In [1]:
%pwd

'/Users/devansh3/Desktop/END_TO_END PROJECTS/Medical_chatbot_GenAI/research'

In [2]:
import os
os.chdir("../")

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf_file(data='data/')

In [6]:
#extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size =500,
                                                 chunk_overlap=20)
    
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [33]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5859

In [27]:
from langchain.embeddings import HuggingFaceEmbeddings

In [28]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

/var/folders/dh/43b_z8dx5_l1rxy05jv622x40000gn/T/ipykernel_73354/1548306741.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
/opt/anaconda3/envs/MED/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [32]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name=index_name,
    embedding = embeddings,
)

In [34]:
# Load Existing Index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [35]:
docsearch

In [38]:
retriever = docsearch.as_retriever(search_type = "similarity",
                                   search_kwargs={"k":3})

In [43]:
retrieved_docs = retriever.invoke("What is Acne?")

In [45]:
retrieved_docs

[Document(metadata={'page': 39.0, 'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 38.0, 'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 37.0, 'source': 'data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for 

In [46]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [58]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question."
    "If the answer is not in the context, say exactly: 'I don't know.'\n"
    "Do not use prior knowledge."
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [59]:
qa_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

In [60]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response['answer'])



Acne is a common skin disease characterized by pimples on the face, chest, and back.


In [61]:
response = rag_chain.invoke({"input": "What is ROMAN?"})
print(response['answer'])



I don't know.
